In [1]:
import numpy       as np
import pandas      as pd
import scipy.stats as stats

In [2]:
data = pd.read_csv('E:\Data Science\Mathematics and Statistics for Data Science\Jupyter Lab\Test\Data\One way ANOVA.txt', sep = '\t')
print(data)

      A     B      C     D
0    93  85.0  100.0  96.0
1   120  45.0   75.0  58.0
2    65  80.0   65.0  95.0
3   105  28.0   40.0  90.0
4   115  75.0   73.0  65.0
5    82  70.0   65.0  80.0
6    99  65.0   50.0  85.0
7    87  55.0   30.0  95.0
8   100  50.0   45.0  82.0
9    90  40.0   50.0   NaN
10   78   NaN   45.0   NaN
11   95   NaN   55.0   NaN
12   93   NaN    NaN   NaN
13   88   NaN    NaN   NaN
14  110   NaN    NaN   NaN


In [7]:
## Xác định k nhóm là k cột trong tập dữ liệu
# groupsA = list(d.columns.values) # array
k      = len(data.columns)
groups = list(data.columns) # (k column headers)

## Tạo k mẫu (loại bỏ giá trị NaN trong các mẫu)
samples = []
for j in range(k):
    sample = [x for x in data[groups[j]] if pd.notnull(x)]
    samples.append(sample)
    
## Số phần tử của mỗi nhóm
sizes = np.zeros(k)
for j in range(k):
    sizes[j] = np.size(samples[j])

In [11]:
# Với  = 0.05, hãy kiểm định giả thuyết H0: “Các quần thể có cùng phương sai.” 

w, pvalue = stats.levene(samples[0], samples[1], samples[2], samples[3])
print(w, pvalue)

if pvalue > 0.05:
    print('không bác bỏ H0: các mẫu từ các quần thể có phương sai bằng nhau.')
else:
    print('bác bỏ H0: các mẫu từ các quần thể có phương sai bằng nhau.')
# p-value > 0.05 => không bác bỏ H0: các mẫu từ các quần thể có phương sai bằng nhau.

0.7977710039415561 0.5020818274234007
không bác bỏ H0: các mẫu từ các quần thể có phương sai bằng nhau.


In [12]:
#Với  = 0.05, hãy cho kết luận về giả thuyết H0: “Các quần thể có cùng giá trị trung bình.”
#bằng 2 phương pháp:

#a) Tính toán truyền thống

##------------------------------------------------------------------------------
print('----------- Cách 1: Tính toán "truyền thống" theo các công thức ---------')
##------------------------------------------------------------------------------    
## Giá trị trung bình của mỗi mẫu
means = np.zeros(k)
for j in range(k):
    means[j] = np.mean(samples[j])

## Giá trị trung bình của tất cả các mẫu
meanT = np.mean(means)

## Các đại lượng BETWEEN groups: SSB, dfB
SSB = 0
for j in range(k):
    SSB += sizes[j] * np.power((means[j] - meanT), 2)
dfB = (k - 1)
print('SSB           : %.4f' % SSB)
print('dfB           : %d' % dfB)

## Các đại lượng WITHIN groups: SSW, dfW
SSW = 0
for j in range(k):
    SSWj = 0
    for i in range(int(sizes[j])):
        SSWj += np.power(samples[j][i] - means[j], 2)
    SSW += SSWj
dfW = int(np.sum(sizes) - k)
print('SSW           : %.4f' % SSW)
print('dfW           : %d' % dfW)

## Trị thống kê: F statistics
F = (SSB / dfB) / (SSW / dfW)
print('F statistic   : %.4f' % F)

## Giá trị tới hạn
##------------------------------------------------------------------------------
## Hàm scipy.stats.f.ppf(q, dfn, dfd) xác định giá trị tới hạn
##    q  : confidence level     (1 - alpha)
##    dfn: tử số (numerator)    dfB (BETWEEN groups)
##    dfd: mẫu số (denominator) dfW (WITHIN groups)
##
## Hàm scipy.stats.f.cdf(crit, dfn, dfd) xác định confidence level (1 - alpha)
##------------------------------------------------------------------------------
critical_value = stats.f.ppf(q=1 - alpha, dfn=dfB, dfd=dfW)
print('Critical value: %.4f' % critical_value)
conf_level = stats.f.cdf(critical_value, dfn=dfB, dfd=dfW)

if (F < critical_value):
    print('(F <  critical value) ==> ACCEPT the H0 that the means are equal.')
else:
    print('(F >= critical value) ==> REJECT the H0 that the means are equal.')


----------- Cách 1: Tính toán "truyền thống" theo các công thức ---------
SSB           : 12486.4848
dfB           : 3
SSW           : 11464.5722
dfW           : 42
F statistic   : 15.2479
Critical value: 2.8270
(F >= critical value) ==> REJECT the H0 that the means are equal.


In [5]:
## b) Dùng các hàm thống kê có sẵn. 
print('----------- Cách 2: Tính toán theo các hàm thống kê ---------')
##------------------------------------------------------------------------------  
alpha = 0.05
dfB = (k - 1)
dfW = int(np.sum(sizes) - k)

critical_value = stats.f.ppf(q = 1 - alpha, dfn = dfB, dfd = dfW)
print('Critical value: %.4f' %critical_value)

fvalue, pvalue = stats.f_oneway(samples[0], samples[1], samples[2], samples[3])

print('p-value       : %.4f' %pvalue)
print('F statistic   : %.4f' %fvalue)

if (fvalue < critical_value):
    print('(F <  critical value) ==> ACCEPT the H0 that the means are equal.')
else:
    print('(F >= critical value) ==> REJECT the H0 that the means are equal.')

----------- Cách 2: Tính toán theo các hàm thống kê ---------
Critical value: 2.8270
p-value       : 0.0000
F statistic   : 15.1391
(F >= critical value) ==> REJECT the H0 that the means are equal.
